In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
finance_news = pd.read_excel('finance_data/SmoothNLP专栏资讯数据集样本10k.xlsx')
finance_news = finance_news.drop(columns = ['Unnamed: 0'])

In [9]:
finance_news.columns

Index(['title', 'content', 'pub_ts'], dtype='object')

In [10]:
finance_news['content'].head()

0    文/易北辰\n在新零售业态当中，无人货架启动和运营成本貌似最低，主要面向2亿白领人群的上班时...
1    \n\n\n\n导读：2018年是网易游戏发展进程当中最重要的一年，甚至比之2015年全面进...
2    \n \n　　文/于斌\n　　其实这句话是个伪命题。按理说，我们早已进入了“扫码取款”的时代...
3    \n      67岁的王石，终究还是回到了他熟悉的商业领域这一战场。在此前一段时间，王石似...
4    \n\n文 | 草原骑士\n来源 | 智能相对论（aixdlun）\n\n1954年，美国成...
Name: content, dtype: object

In [ ]:
for index, content in enumerate(finance_news['content']):
    if '三个第一的背后：“移动搜索 应用商店”双核驱动模式' in content:
        print(content)
        print(index)
        break

In [ ]:
finance_news['content'].iloc[9]

In [12]:
import jieba
jieba.enable_paddle()

Paddle enabled successfully......


In [13]:
tokens = jieba.cut("所以，整个过程有点像勘探。首先是有人发起，然后有千百人买了镐和铁锹跟进。有人会满载而归，但看走眼的人也会空手回来。"
                   ,cut_all=False)

In [3]:
from zhon.hanzi import punctuation as punctuation_zh
from string import punctuation
from words_utils import tokenization as finance_token

In [4]:
stop_words = finance_token.read_stop_words()
tokenizer = finance_token.JiebaTokenizer(stop_words, [punctuation, punctuation_zh], vocab = 'finance_data/vocab')

In [5]:
all_token_length = []
with open('finance_data/data.txt','r', encoding='UTF-8') as file:
    for line in file:
        line = line.strip()
        if len(line) == 0:
            continue
        tokens = tokenizer.tokenize(line)
        all_token_length.append(len(tokens))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.584 seconds.
Prefix dict has been built successfully.


In [11]:
all_token_length = pd.DataFrame(all_token_length)

In [13]:
all_token_length.describe()

,0
count,296933.000000
mean,25.683282
std,22.148801
min,0.000000
25%,10.000000
50%,23.000000
75%,37.000000
max,1902.000000


In [38]:
all_token_length.quantile(q = 0.80)

0    40.0
Name: 0.8, dtype: float64

In [40]:
40 * 0.15

6.0

In [43]:
(20 / 512) * 80

3.125

In [21]:
from dataset import make_pretrain_dataset
import json

In [26]:
train_config = {
    'input_files':'processed_data/train.tf_record',
    'meta_data_file_path':'processed_data/train_meta_data'
}

In [27]:
data_path = train_config['input_files']
with open(train_config['meta_data_file_path'], 'r') as meta_data:
    train_meta_data = json.load(meta_data)

max_seq_length = train_meta_data['max_seq_length']
max_predictions_per_seq = train_meta_data['max_predictions_per_seq']

In [28]:
batch_size = 32
dataset = make_pretrain_dataset(data_path,
                        max_seq_length,
                        max_predictions_per_seq,
                        batch_size=batch_size,
                        is_training=True,
                        input_pipeline_context=None)

In [ ]:
for x, y in dataset:
    print(x)
    break